In [83]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/harshitajain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harshitajain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/harshitajain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [84]:

def extract_keywords(text):
    # Initialize spell checker and lemmatizer
    spell = SpellChecker()
    lemmatizer = WordNetLemmatizer()

    # Set up English stopwords
    stop_words = set(stopwords.words('english'))

    # Tokenize the text
    words = word_tokenize(text)

    # Spell correction, lemmatization and filtering
    corrected_words = [spell.correction(word) for word in words if word.isalpha()]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in corrected_words]
    keywords = [word for word in lemmatized_words if word not in stop_words]

    return keywords

# Example usage
text = "This is a very juicxy and delicious apple."
print(extract_keywords(text))



['This', 'juicy', 'delicious', 'apple']


In [85]:
df=pd.read_csv('clean.csv')

In [86]:
df

,Unnamed: 0,RecipeId,Name,AuthorId_x,AuthorName_x,CookTime,PrepTime,TotalTime,Description,RecipeCategory,...,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeInstructions,ReviewId,AuthorId_y,AuthorName_y,Rating,Review
0,0,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,This is from one of my first Good House Keepi...,Beverages,...,81.5,0.4,77.2,0.3,"c(""Into a 1 quart Jar with tight fitting lid, ...",34854,59064,Missy Wombat,4.333333,My favourite lemonade recipe is essentially th...
1,1,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,Make and share this Cabbage Soup recipe from F...,Vegetable,...,25.1,4.8,17.7,4.3,"c(""Mix everything together and bring to a boil...",46368,71084,smurfy57,2.666667,This is a very good soup. We served ours warm...
2,2,44,Warm Chicken A La King,1596,Joan Edington,PT3M,PT35M,PT38M,I copied this one out of a friend's book so ma...,Chicken,...,29.1,3.1,5.0,45.3,"c(""Melt 1 1/2 ozs butter, add the flour and co...",14,2085,Tony Small,4.545455,An excellent dish.It is a great meal I put it ...
3,3,49,Chicken Breasts Lombardi,174711,Queen Dragon Mom,PT30M,PT45M,PT1H15M,Make and share this Chicken Breasts Lombardi r...,Chicken Breast,...,13.7,0.6,2.0,57.9,"c(""Cook mushrooms in 2 tbsp butter in a large ...",17145,19893,Luke Meakins,4.350000,I give this recipe a good to high rating becau...
4,4,54,Carrot Cake,1535,Marg CaymanDesigns,PT50M,PT45M,PT1H35M,This is one of the few recipes my husband ever...,Dessert,...,67.0,1.6,47.9,5.0,"c(""Beat together the eggs, oil, and white suga...",74639,86318,mianbao,4.235294,"I made this according to the recipe, with no s..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52281,52281,536843,Sheet Pan Turkey Caprese Meatballs With Rosema...,2001112113,Jonathan Melendez,PT40M,PT10M,PT50M,Description: Try these turkey caprese meatball...,Potato,...,33.4,4.1,3.6,37.4,"c(""Preheat oven to 400 degrees F. Line a bakin...",2063253,1802936715,Lv2cook,4.000000,"Sorry to say, it was not good at all. It had n..."
52282,52282,537084,Simple Rotisserie Chicken Tortilla Soup,2002285039,norasingley,PT9M,PT6M,PT15M,Pick a rotisserie chicken that has been rubbed...,< 30 Mins,...,31.2,8.0,4.2,73.7,"c(""Preheat a large pot over high heat. Add oli...",2067886,2002003038,Carol G.,3.666667,Perfect flavor! I followed this recipe to the ...
52283,52283,538991,Nif's Easy Korean Beef and Rice,65502,Nif_H,PT15M,PT5M,PT20M,This is an easy go-to staple in our house! Spi...,Meat,...,186.8,3.8,27.9,50.3,"c(""Heat a large frying pan over medium heat. B...",2073620,1052873,sheepdoc,4.857143,Super easy and tasty. Made rice in the instant...
52284,52284,539090,Cuban Mojo Potatoes,47559,PanNan,PT45M,PT15M,PT1H,Make and share this Cuban Mojo Potatoes recipe...,Cuban,...,40.4,4.3,3.2,4.1,"c(""Lay a sheet of aluminum foil on the counter...",2074314,227978,Northwestgal,4.666667,Very tasty potatoes! It was nice to have them ...


In [87]:
df = df.iloc[:1000]  # Keep the first 100 rows
df

,Unnamed: 0,RecipeId,Name,AuthorId_x,AuthorName_x,CookTime,PrepTime,TotalTime,Description,RecipeCategory,...,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeInstructions,ReviewId,AuthorId_y,AuthorName_y,Rating,Review
0,0,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,This is from one of my first Good House Keepi...,Beverages,...,81.5,0.4,77.2,0.3,"c(""Into a 1 quart Jar with tight fitting lid, ...",34854,59064,Missy Wombat,4.333333,My favourite lemonade recipe is essentially th...
1,1,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,Make and share this Cabbage Soup recipe from F...,Vegetable,...,25.1,4.8,17.7,4.3,"c(""Mix everything together and bring to a boil...",46368,71084,smurfy57,2.666667,This is a very good soup. We served ours warm...
2,2,44,Warm Chicken A La King,1596,Joan Edington,PT3M,PT35M,PT38M,I copied this one out of a friend's book so ma...,Chicken,...,29.1,3.1,5.0,45.3,"c(""Melt 1 1/2 ozs butter, add the flour and co...",14,2085,Tony Small,4.545455,An excellent dish.It is a great meal I put it ...
3,3,49,Chicken Breasts Lombardi,174711,Queen Dragon Mom,PT30M,PT45M,PT1H15M,Make and share this Chicken Breasts Lombardi r...,Chicken Breast,...,13.7,0.6,2.0,57.9,"c(""Cook mushrooms in 2 tbsp butter in a large ...",17145,19893,Luke Meakins,4.350000,I give this recipe a good to high rating becau...
4,4,54,Carrot Cake,1535,Marg CaymanDesigns,PT50M,PT45M,PT1H35M,This is one of the few recipes my husband ever...,Dessert,...,67.0,1.6,47.9,5.0,"c(""Beat together the eggs, oil, and white suga...",74639,86318,mianbao,4.235294,"I made this according to the recipe, with no s..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,7449,Buttered Fried Parsnips,53425,Jenny Sanders,PT45M,PT10M,PT55M,Parsnips are one of the vegetables I think I o...,Vegetable,...,41.0,11.2,10.9,2.9,"c(""Wash, trim and scrape the parsnips."", ""Cut ...",36355,61386,Teasertoo,5.000000,These are excellent!!! Everyone at the table ...
996,996,7455,Chicken and Mushroom Soup,71324,-Sylvie-,NaN,PT0S,PT0S,"I adopted this recipe from Recipezaar, but I h...",Chicken Breast,...,11.9,0.5,5.1,31.0,"c(""Thinly slice the chicken breast meat."", ""Br...",2003,4470,Bergy,4.555556,I just had this for lunch and enjoyed it. I c...
997,997,7461,Peanut Butter Pie V,56112,Jellyqueen,PT20M,PT10M,PT30M,"This is an adopted recipe, but it is just like...",Pie,...,48.6,2.8,34.1,12.5,"c(""Whip cream cheese until fluffy."", ""Add the ...",353970,362788,moski2002,4.857143,"SO easy to make, sweet but not too sweet. On ..."
998,998,7465,Lasagna,87236,truebrit,NaN,PT0S,PT1H,Make and share this Lasagna recipe from Food.com.,Meat,...,36.2,3.7,9.8,41.5,"c(""Saute bacon, crushed garlic, and onion in h...",584,7207,Christi Gilmore,4.611111,This is the best!I made this for my parents 25...


In [88]:
df['reviewkeyword']=df['Review']

/var/folders/x0/nwzkvw4x5g55cyxllzlc0gkm0000gn/T/ipykernel_5416/3814545947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewkeyword']=df['Review']


In [89]:
df['reviewkeyword'][0]

"My favourite lemonade recipe is essentially the same as this. Allowing the sugar to absorb the lemon oils is what makes it so delicious. The syrup is quite concentrated so a little goes a long way. I usually use the zest of 4 lemons, their juice and 1 cup of boiling water and 1 cup sugar. It's good old fashioned lemonade. Use mineral water to dilute the syrup and you get a great fizzy drink.complicated recipe but cool.The taste was excellent, and it really wasn't difficult to make at all.  My stepdaughter and I did this one afternoon, and everything turned out great; the only change we made was to add more hot water to dissolve the sugar; the amount called for was not quite enough.  Very good taste, especially with the lemon zest.  Excellent recipe we'll use often.I made this with limes instead of lemosn and it was the best limeade i've ever had! sooo good!Really nice lemonade. Used a potato peeler to get the zest off.  No white comes off if you do it in a light pressure zig zag.  Aft

In [90]:
df.isna().sum()

Unnamed: 0                      0
RecipeId                        0
Name                            0
AuthorId_x                      0
AuthorName_x                    0
CookTime                      380
PrepTime                        0
TotalTime                       0
Description                     0
RecipeCategory                  0
Keywords                       14
RecipeIngredientQuantities      0
RecipeIngredientParts           0
AggregatedRating                0
ReviewCount                     0
Calories                        0
FatContent                      0
SaturatedFatContent             0
CholesterolContent              0
SodiumContent                   0
CarbohydrateContent             0
FiberContent                    0
SugarContent                    0
ProteinContent                  0
RecipeInstructions              0
ReviewId                        0
AuthorId_y                      0
AuthorName_y                    0
Rating                          0
Review        

In [ ]:
df['CookTime'] = df['CookTime'].fillna('').astype(str)
df['RecipeCategory'] = df['RecipeCategory'].fillna('').astype(str)
df['Keywords'] = df['Keywords'].fillna('').astype(str)
df['AggregatedRating'] = df['AggregatedRating'].fillna('').astype(str)


In [ ]:
na_counts = df.isna().sum()

# Output the result
print(na_counts)

In [ ]:
print(df.dtypes)

In [92]:
def shorten_text(text, max_words=20):
    words = text.split()
    if len(words) > max_words:
        return ' '.join(words[:max_words]) + '...'  # Shorten and add ellipsis to indicate truncation
    return text

# Apply the function to the 'text' column
df['Review'] = df['Review'].apply(shorten_text)

/var/folders/x0/nwzkvw4x5g55cyxllzlc0gkm0000gn/T/ipykernel_5416/214175553.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Review'] = df['Review'].apply(shorten_text)


In [93]:
df['Review'][0]

'My favourite lemonade recipe is essentially the same as this. Allowing the sugar to absorb the lemon oils is what...'

In [102]:

from sentence_transformers import SentenceTransformer
import numpy as np

# Load the pre-trained Sentence-BERT model
model = SentenceTransformer('bert-base-nli-mean-tokens')# Example usage
input_text = " chocolate banana bread"
review_texts = df['Review'].astype(str).tolist()  # Assuming df is your DataFrame
# Get embeddings
input_embedding = model.encode([input_text])  # Encode input text
review_embeddings = model.encode(review_texts)  # Encode all reviews
# Calculate similarities using cosine similarity

# Check shapes for debugging
print("Shape of input_embedding:", input_embedding.shape)
print("Shape of review_embeddings:", review_embeddings.shape)

similarities = cosine_similarity(input_embedding, review_embeddings)  # Should be correct

# Find the index of the most similar review
most_similar_index = np.argmax(similarities)  # Should return the index of the highest value in similarities

# Get the most similar review
most_similar_review = df.iloc[most_similar_index]['Review']
most_similar_review_recipeid = df.iloc[most_similar_index]['RecipeId']
print("Most similar review:", most_similar_review)
print("Recipe Id", most_similar_review_recipeid)

Shape of input_embedding: (1, 768)
Shape of review_embeddings: (1000, 768)
Most similar review: Substituted whole grain bread. Used organic banana, natural (unhydrogenated) peanut butter. I saved time, utensils and steps by mashing the...
Recipe Id 5465


In [103]:
# List of recipe IDs you are interested in
recipe_ids = [5465]

# Filter the DataFrame to only include rows with these recipe IDs
filtered_df = df[df['RecipeId'].isin(recipe_ids)]

# Display the filtered DataFrame
filtered_df['RecipeIngredientParts']
filtered_df['Name']


845    Elvis Presley's Grilled Peanut Butter and Bana...
Name: Name, dtype: object

In [112]:
import fasttext

# Example: Train a FastText model on a file containing a list of recipes.
# Each recipe should be on a new line, and ingredients should be space-separated.
model = fasttext.train_unsupervised('recipes.txt', model='skipgram')

# After the model is trained, you can find similar ingredients
# For example, finding replacements for "almond"
print(model.get_nearest_neighbors('onion'))
print(model.words)

Read 0M words
Number of words:  1027
Number of labels: 0


[(0.9999704360961914, 'onions'), (0.9999655485153198, 'onion,'), (0.9999518990516663, 'onions,'), (0.9998708963394165, 'crumbled'), (0.9998692870140076, 'parsley'), (0.9998672604560852, 'Coarsely'), (0.9998645782470703, 'cinnamon'), (0.9998636841773987, 'coarsely'), (0.9998621940612793, 'speed'), (0.9998605251312256, 'parsley.')]
['</s>', 'the', 'and', '1', 'a', 'to', 'in', 'c', 'with', 'until', 'ts', 'of', '2', 'for', 'or', '1/2', 'tb', 'Add', 'into', 'on', 'is', 'over', '1/4', '4', 'minutes.', 'Salt', '3', 'pepper', 'chopped', 'Recipe', 'at', 'cheese', 'from', 'sugar', 'Stir', 'about', 'water', 'heat', 'mixture', 'Categories:', 'Yield:', 'Title:', 'Servings', 'via', 'add', 'minutes', 'Meal-Master', 'cook', '(tm)', 'v8.05', 'large', 'MMMMM-----', 'butter', 'MMMMM', 'oil', 'oven', 'sauce', 'remaining', 'it', 'dough', 'small', 'by', '--', 'baking', 'top', 'salt', 'stir', 'are', 'In', 'Bake', 'medium', '6', 'Remove', 'then', 'Cover', 'simmer', 'Serve', 'cream', 'ingredients', 'bowl', '5'

Progress: 100.0% words/sec/thread:  315856 lr:  0.000000 avg.loss:  2.933394 ETA:   0h 0m 0s


In [106]:
# Sample recipes data
recipes = [
    "flour sugar butter egg vanilla",
    "tomato basil mozzarella garlic olive_oil",
    "chicken olive_oil garlic lemon parsley",
    "beef onion garlic carrot potato tomato",
    "salmon lemon garlic butter parsley"
]

# Writing the data to a text file
with open("recipes.txt", "w") as file:
    for recipe in recipes:
        file.write(recipe + "\n")
